# Описание бизнес задачи

**Бизнес задача***:	увеличить конверсию	продаж туров

**Гипотеза**:	баннеры предиктивной аналитики увеличивают конверсию продаж туров				

**Маркетинговые задачи**:	
* разработать продающие баннеры	  			
* разместить баннеры на разных точчках сайта				

**Аналитические задачи**:	
* Обеспечить метрики просмотра баннера: id, время сопостовимые с client_id и user_id (при наличии)				
* Обеспечить метрики клика баннера: id, время сопостовимые с client_id и user_id (при наличии)				
* Обеспечить полную информацию о проданных турах: id  заказа, время заказа, сумма заказа, client_id				
* Построить воронку продаж туров по этапам: просмотр-клик-заказ				
* У воронки должен быть фильтр по месту продажи тура

## 1. Подключение к базе, изучение состава информации

In [3]:
# загрузка пакетов для работы с SQL запросами
import pandas as pd
import psycopg2
import psycopg2.extras

In [ ]:
connect_param=f"dbname={dbname1} user={user1} host={host1} password={password1} port={port1}"

In [16]:
# Создаем функцию SQL_read(connect_param,query_) - SQL-запрос для выбора данных, где на входе:
# connect_param - параметры запроса
# query_ - текст запроса

def SQL_read(connect_param,query_):
    
    query = query_.format()
    
    conn = psycopg2.connect(connect_param)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data  

In [8]:
# снимем ограничение на количество строк и столбцов
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# посмотрим на наличие ключей для соеднения таблиц
query_constraint = """  SELECT 
                            tc.table_name,
                            kcu.column_name,
                            tc.constraint_type,
                            ccu.table_name AS foreign_table_name,
                            ccu.column_name AS foreign_column_name
                        FROM 
                            information_schema.table_constraints tc
                            JOIN information_schema.key_column_usage kcu ON tc.constraint_name = kcu.constraint_name
                            LEFT JOIN information_schema.constraint_column_usage AS ccu ON tc.constraint_name = ccu.constraint_name
                        WHERE 
                            tc.table_schema = 'public'
                            AND (tc.constraint_type = 'PRIMARY KEY' OR tc.constraint_type = 'FOREIGN KEY');
                    """

df_constraint=pd.DataFrame(SQL_read(connect_param, query_constraint))
df_constraint

,table_name,column_name,constraint_type,foreign_table_name,foreign_column_name
0,cid_and_yandex_id,user_pseudo_id,PRIMARY KEY,cid_and_yandex_id,user_pseudo_id


Ключи в базе не размечены

In [18]:
# создадим запрос, который вытаскивает все паля из всех таблиц
query_col = f'''SELECT table_name, column_name
                FROM information_schema.columns
                WHERE table_schema = 'public'
                    '''
df_col=pd.DataFrame(SQL_read(connect_param, query_col))

In [140]:
# выделяем интересующие нас таблицы

A = df_col[df_col['table_name'] == 'appmetrica']
VH = df_col[df_col['table_name'] == 'visits_hits']
O = df_col[df_col['table_name'] == 'orders']
CY =  df_col[df_col['table_name'] == 'cid_and_yandex_id']
C = df_col[df_col['table_name'] == 'clients']
P = df_col[df_col['table_name'] == 'payments']
YH = df_col[df_col['table_name'] == 'ym_hits']
YV = df_col[df_col['table_name'] == 'ym_visits']
Y193 = df_col[df_col['table_name'] == 'yandex_193']
ST = df_col[df_col['table_name'] == 'session_totals']
GGL = df_col[df_col['table_name'] == 'ggl_sheet_costs_conn']
AM = df_col[df_col['table_name'] == 'aso_uid_to_sessions_main']
AU = df_col[df_col['table_name'] == 'aso_uid_to_sessions_update']
AS = df_col[df_col['table_name'] == 'aso_calltouch_usedesk_to_sessions']
AEU = df_col[df_col['table_name'] == 'aso_events_to_sessions_update']
AEM = df_col[df_col['table_name'] == 'aso_events_to_sessions_main']
AC = df_col[df_col['table_name'] == 'advcake_costs_conn']
CT = df_col[df_col['table_name'] == 'calltouch_test2']
MC = df_col[df_col['table_name'] == 'mytarget_cost_conn']
UC = df_col[df_col['table_name'] == 'usedesk_conn']
VK = df_col[df_col['table_name'] == 'vk_costs_conn']
YC = df_col[df_col['table_name'] == 'yd_costs_conn']
UCP = df_col[df_col['table_name'] == 'unique_cid_params']
I = df_col[df_col['table_name'] == 'installations']
W = df_col[df_col['table_name'] == 'web_funnel']
YST = df_col[df_col['table_name'] == 'yandex_session_totals']

#соединяем названия столбцов, смотрим пересекающиеся метрики
df_merged = A.merge(VH, how = 'outer')
df_merged = df_merged.merge(O, how = 'outer')
df_merged = df_merged.merge(CY, how = 'outer')
df_merged = df_merged.merge(C, how = 'outer')
df_merged = df_merged.merge(P, how = 'outer')
df_merged = df_merged.merge(YH, how = 'outer')
df_merged = df_merged.merge(YV, how = 'outer')
df_merged = df_merged.merge(Y193, how = 'outer')
df_merged = df_merged.merge(ST, how = 'outer')
df_merged = df_merged.merge(GGL, how = 'outer')
df_merged = df_merged.merge(AM, how = 'outer')
df_merged = df_merged.merge(AU, how = 'outer')
df_merged = df_merged.merge(AS, how = 'outer')
df_merged = df_merged.merge(AEU, how = 'outer')
df_merged = df_merged.merge(AEM, how = 'outer')
df_merged = df_merged.merge(AC, how = 'outer')
df_merged = df_merged.merge(CT, how = 'outer')
df_merged = df_merged.merge(MC, how = 'outer')
df_merged = df_merged.merge(UC, how = 'outer')
df_merged = df_merged.merge(VK, how = 'outer')
df_merged = df_merged.merge(YC, how = 'outer')
df_merged = df_merged.merge(UCP, how = 'outer')
df_merged = df_merged.merge(I, how = 'outer')
df_merged = df_merged.merge(W, how = 'outer')
df_merged = df_merged.merge(YST, how = 'outer')

df_transposed = df_merged.pivot_table(index='table_name', columns='column_name', aggfunc=lambda x: ', '.join(x)).fillna('')
df_transposed = df_transposed.T
df_transposed = df_transposed.applymap(lambda x: 1 if x == 'table_name, column_name' else '')


## 2. Загрузка таблиц

В ходе изучения базы данных были выявлены таблицы, которые содержат необходиму для построения воронки информацию, это:
1. orders - информация о заказах
2. visits_hits - информация о кликах
3. yandex_session_totals

Универсального алгоритма поиска данных нет. Что можно сделать:
1. Выяснить у заказчика где хранятся необходимые данные в ходе сбора бизнес требований и последующих коммуникаций.  
2. Изучить название таблиц и название признаков, которые они содержат. Чаще всего названия являются информативными.
3. Поэтапно просмотривать данные, хранящиеся в базе. Изучить схему, которую вывели кодом выше и возможности соединения таблиц по ключевым (одинаковым) полям (чаще всего эти поля содержат метку id).

Поскольку кейс является реальным рабочим опытом, информация о содержании таблиц будет скрыта.

In [20]:
# запрос, что бы вывести все данные из таблицы
# с этим нужно быть осторожным, т.к. если таблица тяжелая, а оперативная память у вашего ПК маленькая, работа кода приведет к зависанию браузера, 
# либо его аварийному завершению и все несохраненные даные будут потеряны
query_orders	 = f'''SELECT *
                       FROM public.orders
                    '''
df_orders=pd.DataFrame(SQL_read(connect_param, query_orders))

# далее с df_orders можно работать как с обычным датафреймом, использую любые доступные вам методы для изучения его содержания

In [21]:
# для того, что бы посмотреть структуру данных, но не рисковать при этом оперативной памятью, можно ограничить количество выводимых значений

query_visits_hits = f'''SELECT *
                        FROM public.visits_hits
                        ORDER BY date DESC
                        LIMIT 10
                    '''
df_visits_hits=pd.DataFrame(SQL_read(connect_param, query_visits_hits))

# но важно помнить, что и в переменной df_visits_hits будет храниться только 10 строк данных

In [22]:
query_yandex_session_totals = f""" select
                                        *
                                    from
                                        public.yandex_session_totals
                                    order by
                                        date desc
                                    limit 10"""

df_yandex_session_totals = pd.DataFrame(SQL_read(connect_param, query_yandex_session_totals))


## 3. Разбор итогового запроса

В ходе изучения данных была выявлена **проблема**,  а именно: yandex_session_totals не содержит актуальной информации о просмотрах баннеров.  

**Варианты решения**:  
1. дождаться пока заказчик правильно разметит данные и эта информация попадет в базу, но время - деньги, а время простоя - упущеная прибыль
2. создать фальш - таблицу для демонстрации результата запроса, который будет после корректного занесения данныйх - в этом случае можно избежать простоя в работе, но нужно будет вернуться к доработке запроса, когда актульные данные начнут поступать в базу.

**Как это реализовать?**  
1. Нужно определиться с составом тех данных, которые у нас есть.
Для этого, как ни странно, нужно начать с конца: в любом удобном инструченте создать макет/модель таблицы, которая должна получиться на выходе. Т.е. определить признаки (столбца) и примерно на примере нескольких строк расставить данные в них.
2. Далее попытаться воспроизвести эту таблицу с помощью запроса и оценить:  
2.1. какие данные отсутствуют и из каких они таблиц  
2.2. в нужном ли формате те данные, что у нас в наличии  
3. Определиться фальш таблица будет полностью придуманной, или какие-то данные будут взяты из других таблиц.

Для начала я приведу итоговый запрос и выведу результат, а далее разберу его.

In [23]:
query_result = f""" WITH intermediate_table AS (                                                                    
                                            SELECT 
                                                'искуственная запись_1' AS type,
                                                1662974822267180842 AS client_id,
                                                'Hotel Holiday International' AS hotel,
                                                7642218 AS order_id,
                                                '2023-10-26 23:41:20' as banner_view_time,
                                                'клиент пришел, увидел, кликнул, купил' as comment
                                            UNION ALL
                                            SELECT 
                                                'искуственная запись_2' AS type,
                                                1662974822267180843 AS client_id,                              
                                                'Hedef Resort Hotel' AS hotel,
                                                NULL AS order_id,
                                                '2023-10-26 12:13:50' AS banner_view_time,
                                                'клиент пришел, увидел и ушел' as comment
                                            UNION ALL
                                            SELECT 
                                                'искуственная запись_3' AS type,
                                                1662974822267180844 AS client_id,                              
                                                'Rixos Beldibi - The Land Of Legends Free Access' AS hotel,
                                                NULL AS order_id,
                                                '2023-10-27 23:13:50' AS banner_view_time,
                                                'клиент пришел, увидел и ушел' as comment
                                            UNION ALL
                                            SELECT 
                                                'искуственная запись_4' AS type,                         -- обозначение иск записи
                                                1698071697423008456 AS client_id,                        -- ключ к visit_hits
                                                'TIA Heights Makadi Bay' AS hotel,                       -- отель в брони
                                                7644698 AS order_id,                                     -- ключ к orders
                                                '2023-10-26 23:13:50' as banner_view_time,
                                                'клиент пришел, увидел, кликнул, купил' as comment
                                                
                                        ),
                        visits_result AS (
                                            SELECT
                                                *
                                            FROM
                                                (
                                                SELECT 
                                                    parsed_params->'blockName' AS hotel_name,           -- название баннера
                                                    *                                                   -- все остальные столбцы из visit_hits
                                                FROM 
                                                    visits_hits
                                                WHERE 
                                                    date = '2023-10-26'
                                                    AND ARRAY[296872441,296872590] && goals_id          -- 296872441 просмотр баннера, 296872590 клик в баннер
                                                ORDER BY 
                                                    date_time DESC
                                                ) t 
                                            WHERE 
                                                hotel_name NOT IN ('"Рекомендуем именно вам!"', '"null"', '"undefined"','""')
                                        )
                                        
                                        SELECT 
                                             count(i. client_id) as banner_view,                                        -- просмотры баннера из яндекс тотал в будущем
                                             i.banner_view_time ,                       
                                             coalesce(v.visit_id, 6338621931867600000) as banner_view_visit_id,         -- !!! временная замена для таблицы
                                             CASE
                                                WHEN v.title	 LIKE 'Купить%' THEN 'Поиск'
                                                WHEN v.title	 LIKE '%404%' THEN '404'
                                                WHEN v.title	 LIKE 'Официальный%'  THEN 'Главная'
                                                WHEN v.title is null THEN '404'                                -- !!временно пока нет просмотров
                                            END AS banner_view_URL,
                                             coalesce (v.client_id, 1662974822267180000) as client_id,                  -- !!! временная замена для таблицы
                                             i. hotel,                                                            -- просмотренные отели
                                             count(v.client_id) as banner_click,                                        -- клики в баннер из подзапроса visits_result AS
                                             v.date_time AS banner_click_time,
                                             v.hotel_name,
                                             o.create_date as hotel_booking_time,
                                             i.order_id  AS order_id,                                        -- номер брони из яндекс тотал в будущем
                                             o.orderid_status,
                                             coalesce(o.сatalog_price, 0) as revenue
                                          FROM 
                                            intermediate_table as i 
                                            left JOIN visits_result as v ON v.client_id = i.client_id
                                            left join public.orders as o on i.order_id = o.order_id 
                                         WHERE
                                              (v.date_time IS NULL AND o.create_date IS NULL AND CAST(i.banner_view_time AS timestamp without time zone) IS NOT NULL) -- если только просмотры
                                            OR (CAST(i.banner_view_time AS timestamp without time zone) < v.date_time AND o.create_date IS NULL)                      -- если просмотры и клики
                                            OR (CAST(i.banner_view_time AS timestamp without time zone) < v.date_time AND v.date_time < o.create_date)                -- если бронь
                                                                                      
                                         GROUP BY
                                             i. client_id,                                      -- просмотры баннера из яндекс тотал в будущем
                                             v.client_id,                                              -- клики в баннер из подзапроса visits_result AS
                                             v.visit_id,
                                             i. hotel,                                           -- название баннера
                                             v. hotel_name,                                              -- забронированый отель
                                             i.banner_view_time,
                                             v.date_time,
                                             i.order_id,                                        -- номер брони из яндекс тотал в будущем
                                             o.create_date,
                                             o.orderid_status,
                                             o.сatalog_price,
                                             v. date_time,
                                             v.title
                                        ORDER BY
                                            i.banner_view_time desc
                                     
            """

df_result = pd.DataFrame(SQL_read(connect_param, query_result))
df_result

,banner_view,banner_view_time,banner_view_visit_id,banner_view_url,client_id,hotel,banner_click,banner_click_time,hotel_name,hotel_booking_time,order_id,orderid_status,revenue
0,1,2023-10-27 23:13:50,6338621931867600000,404,1662974822267180000,Rixos Beldibi - The Land Of Legends Free Access,0,NaT,None,NaT,NaN,None,0
1,1,2023-10-26 23:41:20,6338621931867603017,Поиск,1662974822267180842,Hotel Holiday International,1,2023-10-26 23:41:27,Hotel Holiday International,2023-10-27 14:38:00,7642218.0,Оплачено,151663.35
2,1,2023-10-26 23:13:50,6337844802030141700,Поиск,1698071697423008456,TIA Heights Makadi Bay,1,2023-10-26 23:13:54,TIA Heights Makadi Bay,2023-10-28 22:07:00,7644698.0,Отменено,0.00
3,1,2023-10-26 12:13:50,6338621931867600000,404,1662974822267180000,Hedef Resort Hotel,0,NaT,None,NaT,NaN,None,0


Итак, для построения воронки продаж нужна информация:
* об отеле, который был просмотрен - название баннера
* о количестве таких просмотров
* об отеле, по которому произведен клик - название отеля
* о количестве таких кликов
* о бронировании отеля - название отеля, сумма заказа
! Важно учесть действия пользователя во времени, т.е. сначала клик, потом просмотр, потом покупка. Время покупки никак не может быть раньше, времени просмотра для целей составления вронки.

**Например**:   
Баннер посмотрели 10 раз, кликнули 9 раз, купили отель 3 раза  - это и есть воронка заказа, если вы нарисуете ее любым способом, она будет похожа на конус, сужающийся к низу.  
Конверсия такой воронки 33% (3/10).  

Это итоговая, агрегированная информация, которая нам нужна. А что в сырых данных?  

**Просмотр отеля** имеет следующие характеристики:   
* идентификатор клиента, который посмотрел баннер (сложив их количество мы и получим количество просмотров). Важно, что именно клиента, а не юзера, т.е. этот человек еще может быть не авторизирован на сайте И просмотр находится в таблице с просмотрами (в данном случае yandex_session_totals);
* время просмотра баннера
* место, где этот баннер физически находится на сайте (в данном случае это: главная страница, поисковая выдача или страница 404 - эти параметры определены клиентом)
* название отеля (важно помнить, что посетитель сайта может просмотреть несколько отелей, а кликнуть только по одному из них)

**Клик отеля** имеет следующие характеристики:
* идентификатор клиента, который кликнул баннер в таблице с кликами (в данном примере это visits_hits) и именно этот параметр является ключом для соединения с таблицей просмотров (в данном случае yandex_session_totals)
* время клика


**Заказ** или в данном случае бронирование тура имеет следующие характеристики:  
* название отеля
* время бронирования или время создания заказа
* номер заказа - это второй ключ к соединению нужных таблиц
* статус заказа (это доп параментр, о котором клиент не просил, но который всплыл в ходе исследования данных - тур может по разным причинам остаться не оплаченным и таким образом посетитель сайта прошел все этапы воронки, но при этом прибыли это все равно не принесло)
* сумма заказа

Ну вот **первый этап** и пройден. Теперь мы имеем представление, какие нам нужны данные, а значит знаем, что нужно искать.  
Переходим ко **второму**.

При воспроизведении таблиц было выяснено, что для удачного построения воронки в yandex_session_totals должна содержаться следующая информация:  
* идентификатор клиента, который посмотрел баннер - ее нет, но он тот же что и у клика баннера
* название отеля - ее тоже нет и она тоже такая же как у клика (логика заключается в том, что клиент не мог не кликнуть в баннер, не просмотрев его)
* время просмотра баннера - тоже нет, но оно должно быть раньше клика
* номер заказа - единственное, что есть в наличии

Остальные 2 таблицы содержат всю необходимую информацию.  
Очевидно, что фальш таблица будет польностью заменять таблицу yandex_session_totals.  
При этом спроектируем 4 строки - 2 содержат просмотры, которые привели к кликам - возьмем для них идентификаторы из кликов (помним что clien_id по этим событиям совпадает) и 2 строки, которые остались на уровне просмотров - просто придумаем для них информацию.   

Запрос для фальш таблицы будет иметь следующий вид:

In [ ]:
                                           SELECT 
                                                'искуственная запись_1' AS type,
                                                1662974822267180842 AS client_id,
                                                'Hotel Holiday International' AS hotel,
                                                7642218 AS order_id,
                                                '2023-10-26 23:41:20' as banner_view_time,
                                                'клиент пришел, увидел, кликнул, купил' as comment
                                            UNION ALL
                                            SELECT 
                                                'искуственная запись_2' AS type,
                                                1662974822267180843 AS client_id,                              
                                                'Hedef Resort Hotel' AS hotel,
                                                NULL AS order_id,
                                                '2023-10-26 12:13:50' AS banner_view_time,
                                                'клиент пришел, увидел и ушел' as comment
                                            UNION ALL
                                            SELECT 
                                                'искуственная запись_3' AS type,
                                                1662974822267180844 AS client_id,                              
                                                'Rixos Beldibi - The Land Of Legends Free Access' AS hotel,
                                                NULL AS order_id,
                                                '2023-10-27 23:13:50' AS banner_view_time,
                                                'клиент пришел, увидел и ушел' as comment
                                            UNION ALL
                                            SELECT 
                                                'искуственная запись_4' AS type,                         -- обозначение иск записи
                                                1698071697423008456 AS client_id,                        -- ключ к visit_hits
                                                'TIA Heights Makadi Bay' AS hotel,                       -- отель в брони
                                                7644698 AS order_id,                                     -- ключ к orders
                                                '2023-10-26 23:13:50' as banner_view_time,
                                                'клиент пришел, увидел, кликнул, купил' as comment

Поскольку эта информация искусственная, каждую строкунеобходимо создавать отдельно через оператор UNION ALL.   
В первой и последней строке просто доп информация для удобства, она не участвует в в построении запроса. 
При использовании UNION ALL структура данных должна быть полностью соблюдена, и если например нет номера заказа, то необходимо обозначить в этом месте пустое значение (думаю чем пустое значение для целей аналитики отличается от 0 объяснять не нужно).  

Итак итентификаторы строк с заказами - взяты из visits_hits (просто первые 2 записи, где есть заказ), время в этих строках искусстенное - взято из visits_hits и подправлено на несколько секунт раньше, название отеля так же взято из visits_hits.

Информация в строках без заказов полностью выдуманная.  

Далее нам необходимо получить информацию о кликах:

In [ ]:
                                            SELECT
                                                *
                                            FROM
                                                (
                                                SELECT 
                                                    parsed_params->'blockName' AS hotel_name,           -- название баннера
                                                    *                                                   -- все остальные столбцы из visit_hits
                                                FROM 
                                                    visits_hits
                                                WHERE 
                                                    date = '2023-10-26'
                                                    AND ARRAY[296872441,296872590] && goals_id          -- 296872441 просмотр баннера, 296872590 клик в баннер
                                                ORDER BY 
                                                    date_time DESC
                                                ) t 
                                            WHERE 
                                                hotel_name NOT IN ('"Рекомендуем именно вам!"', '"null"', '"undefined"','""')

Для этого нам нужно соединить таблицу visits_hits саму с собой.  
Во внутренней части запроса мы выводи все поля, а так же достаем название отеля из поля parsed_params.  Это поле представляет собой словарь и мы обращаемся к нему по ключу 'blockName'.   
Далее фильтруем для производительности по дате, т.е. только после этой точки была произведена разметка кликов на стороне клиента и обозначаем какие 2 контрольные события нас интересуют через поле goals_id.  

После того как мы вытащили основную информацию, фильтруем ее еще раз, что бы не было не эффективных данных.   
Т.е. если не поставить условие WHERE hotel_name NOT IN ('"Рекомендуем именно вам!"', '"null"', '"undefined"','""') в итоговом запросе выйдет куча пустых строк.    

Эти 2 таблицы у нас будут как вспомогательные.  
Для того, что выбрать нужные данные из временных вспомогательных таблиц, необходимо использовать СТЕ (Common Table Expressions) запросы.   
Суть их состоит в том, что мы сначала группируем нужные нам данные во временные таблицы, а затем выбираем уже из временных таблиц необходимую информацию.  Так же их можно соеднять и с любыми другими таблицами базы по ключам, главное эти ключи предусмотреть.  

Соединение итогового запроса происходит слева. Это необходимо для того, что бы оставалась все этапы воронки, а не только те, где есть и просмотры, и клики, и визиты.   
Сам запрос приведен выше, рассмотрим только его итоговую часть после составления вспомогательных таблиц.

In [ ]:
                                           SELECT 
                                             count(i. client_id) as banner_view,                                        -- просмотры баннера из яндекс тотал в будущем
                                             i.banner_view_time ,                       
                                             coalesce(v.visit_id, 6338621931867600000) as banner_view_visit_id,         -- !!! временная замена для таблицы
                                             CASE
                                                WHEN v.title	 LIKE 'Купить%' THEN 'Поиск'
                                                WHEN v.title	 LIKE '%404%' THEN '404'
                                                WHEN v.title	 LIKE 'Официальный%'  THEN 'Главная'
                                                WHEN v.title is null THEN '404'                                          -- !!временно пока нет просмотров
                                            END AS banner_view_URL,
                                             coalesce (v.client_id, 1662974822267180000) as client_id,                   -- !!! временная замена для таблицы
                                             i. hotel,                                                                   -- просмотренные отели
                                             count(v.client_id) as banner_click,                                         -- клики в баннер из подзапроса visits_result 
                                             v.date_time AS banner_click_time,
                                             v.hotel_name,
                                             o.create_date as hotel_booking_time,
                                             i.order_id  AS order_id,                                                    -- номер брони из яндекс тотал в будущем
                                             o.orderid_status,
                                             coalesce(o.сatalog_price, 0) as revenue
                                          FROM 
                                            intermediate_table as i 
                                            left JOIN visits_result as v ON v.client_id = i.client_id
                                            left JOIN public.orders as o on i.order_id = o.order_id 
                                         WHERE
                                              (v.date_time IS NULL AND o.create_date IS NULL AND CAST(i.banner_view_time AS timestamp without time zone) IS NOT NULL) -- если только просмотры
                                            OR (CAST(i.banner_view_time AS timestamp without time zone) < v.date_time AND o.create_date IS NULL)                      -- если просмотры и клики
                                            OR (CAST(i.banner_view_time AS timestamp without time zone) < v.date_time AND v.date_time < o.create_date)                -- если бронь
                                                                                      
                                         GROUP BY
                                             i. client_id,                                              
                                             v.client_id,                                               
                                             v.visit_id,
                                             i. hotel,                                                  
                                             v. hotel_name,                                             
                                             i.banner_view_time,
                                             v.date_time,
                                             i.order_id,                                                
                                             o.create_date,
                                             o.orderid_status,
                                             o.сatalog_price,
                                             v. date_time,
                                             v.title
                                        ORDER BY
                                            i.banner_view_time desc

In [ ]:
Рассмотри сначала соединение:  
FROM 
    intermediate_table as i 
    left JOIN visits_result as v ON v.client_id = i.client_id
    left JOIN public.orders as o on i.order_id = o.order_id   

как я и писала выше, оно происходит слева: ко всем просмотрам мы присоединяем все клики по client_id, если оставить внутренне соединение (обычный JOIN), тогда остануться только те строки где есть и просмотры, и клики

In [ ]:
Далее обращу ваше внимание на разметку для фильтра:  
CASE
     WHEN v.title	 LIKE 'Купить%' THEN 'Поиск'
     WHEN v.title	 LIKE '%404%' THEN '404'
     WHEN v.title	 LIKE 'Официальный%'  THEN 'Главная'
     WHEN v.title is null THEN '404'                                          -- !!временно пока нет просмотров
END AS banner_view_URL,

Эта часть кода создает дополнительный столбец banner_view_URL, куда записывает 'Поиск', '404' или 'Главная' в зависимости от указанного условия.
Оператор LIKE отбирает в столбце v.title по заданным фильтрам информацию, % - обозначает что до или после клучевой фразы могут быть любые значения.
Последняя строка добавлена, что бы в итоговом запросе это поле не оставалось пустым, т.к. логично если баннер просмотрен, то он проспотрен в определенной части сайта и пустых значений тут быть не может. 

Что еще есть **интересного**:  
coalesce(v.visit_id, 6338621931867600000)    
coalesce(o.сatalog_price, 0)  


Оператор coalesce() заменяет пустое значение на указанное после запятой.
Зачем его тут применять?  
Во-первых, визуально красивее, когда вместо null стоит 0 в колонке с выручкой, кроме того это помогает избегать ошибок обработки данных в тех сервисах, где null не воспринимется и его наличие выдает ошибку.   
Во-вторых, в приведенном примере, даже если клика не было client_id существует (а мы помним, что это у нас просмотр и ключ) и не может быть пустым, по-этому тут заложены искусственные данные пользователей, которые только посмотрели.

In [ ]:
Напоследок остановлюсь еще на условии по времени. Мы помним, что сначала просмотр, потом клик, потом заказ. 

WHERE
     (v.date_time IS NULL AND o.create_date IS NULL AND CAST(i.banner_view_time AS timestamp without time zone) IS NOT NULL) -- если только просмотры
    OR (CAST(i.banner_view_time AS timestamp without time zone) < v.date_time AND o.create_date IS NULL)                      -- если просмотры и клики
    OR (CAST(i.banner_view_time AS timestamp without time zone) < v.date_time AND v.date_time < o.create_date)                -- если бронь

Но если эти условия просто выставить по очередно, опять же останутся только те данные, где есть и простмотры, и клики, и заказы.  
Что бы выйти из этой ситуации нужно рассмотреть вариант 1 когда есть только просмотры, далее вариант 2 когда просмотры раньше кликов, но нет заказов и, наконец, вариант 3, когда все есть. Ну и соединить это все хозяйство в одно условие.

Оператор CAST в данном случае применен для искусственных данных, что бы обозначить это это формат timestamp without time zone, когда будут реальные данные этого оператора не будет.    

Далее после написания запроса в качестве отчета была произведена визуализация в Power BI с демонстрацией возможносте